In [1]:
{-# LANGUAGE InstanceSigs #-}

In [2]:
class Monad m => Writer m where
    write :: String -> m ()

In [3]:
newtype W a = W (a, String)

instance Functor W where
    fmap :: (a -> b) -> W a -> W b
    fmap f (W (a, s)) = W (f a, s)

instance Applicative W where
    pure :: a -> W a
    pure a = W (a, "")

    (<*>) :: W (a -> b) -> W a -> W b
    (<*>) (W (f, s)) (W (a, s')) = W (f a, s ++ s')

instance Monad W where
    (>>=) :: W a -> (a -> W b) -> W b
    (>>=) (W (a, s)) f = let W (b, s') = f a in W (b, s ++ s')

instance Writer W where
    write s = W ((), s)

output :: W a -> String
output (W (a, s)) = s

In [4]:
import Control.Monad.Trans.Class

In [5]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}

import Control.Monad.Trans.Cont

newtype C m a = C (ContT m Action a) deriving (Functor, Applicative, Monad)

data Action m
    = Atom (m (Action m))
    | Fork (Action m) (Action m)
    | Stop

In [6]:
action :: Monad m => C m a -> Action m
action (C (ContT m)) = m $ \_ -> Stop

atom :: Monad m => m a -> C m a
atom m = C $ ContT $ \c -> Atom $ c <$> m

stop :: Monad m => C m a
stop = C $ ContT $ \_ -> Stop

par :: Monad m => C m a -> C m a -> C m a
par (C (ContT m1)) (C (ContT m2)) = C $ ContT $ \c -> Fork (m1 c) (m2 c)

fork :: Monad m => C m a -> C m ()
fork m = C $ ContT $ \c -> Fork (action m) (c ())

instance MonadTrans C where
    lift :: Monad m => m a -> C m a
    lift = atom

In [7]:
round :: Monad m => [Action m] -> m ()
round [] = pure ()
round (a:as) = case a of
    Atom am -> do
        a' <- am
        round (as ++ [a'])
    Fork a1 a2 -> round (as ++ [a1, a2])
    Stop -> round as

run :: Monad m => C m a -> m ()
run m = round [action m]

In [8]:
instance Writer m => Writer (C m) where
    write s = lift (write s)

loop :: Writer m => String -> m ()
loop s = do
    write s
    loop s

In [9]:
example :: Writer m => C m ()
example = do
    write "start!"
    fork (loop "fish")
    loop "cat"

-- output (run example)

In [10]:
instance Writer m => Writer (C m) where
    write [] = pure ()
    write (c:s) = do
        lift (write [c])
        write s

-- output (run example)

In [11]:
import Control.Applicative
import Data.Foldable (asum)

In [12]:
instance Monad m => Alternative (C m) where
    (<|>) :: C m a -> C m a -> C m a
    (<|>) = par
    
    empty :: C m a
    empty = stop

merge :: [String] -> String
merge = output . run . asum . map write

In [13]:
import Data.IORef

type MVar a = IORef (Maybe a)

newMVar :: C IO (MVar a)
newMVar = lift $ newIORef Nothing

writeMVar :: MVar a -> a -> C IO ()
writeMVar v a = lift $ writeIORef v (Just a)

takeVar :: MVar a -> IO (Maybe a)
takeVar v = do
    am <- readIORef v
    writeIORef v Nothing
    pure am

readMVar :: MVar a -> C IO a
readMVar v = do
    am <- lift (takeVar v)
    case am of
        Nothing -> readMVar v
        Just a  -> pure a 